Training a model can be time consuming. TensorFlow offers a several distributed strategies to enable speeding up of model training, applicable to both large models and large datasets. This model demonstrates the TensorFlow distributed API.

## Getting ready...

The TensorFlow distributed API allows us to distribute the training by replicating the model into different nodes and training on different subsets of data. Each strategy supports a hardware platform (multiple GPU, multiple machines, or TPUs) and uses either a synchronous or asynchronous training strategy. In synchronous training, each worker trains over different batches of data and aggregates their gradients at each step. In asynchronous mode, each worker is independently training over the data and hte variables are updated asynchronously. Allegedly, TensorFlow will soon support model parallelism. This paradigm is used when the model is too large to fit on a single device and needs to be distributed over many devices. In this recipe, 

## How to do it...

1. Load required libraries

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

2. Create two virtual GPUs:

In [4]:
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpu_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024),
                                                                                 tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        # Memory growth cannot be modified after GPU has been initialized
        print(e)

3. We load the MNIST dataset with the tensorflow_datasets API

In [5]:
datasets, info = tfds.load('mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

4. Prepare the data

In [6]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

mnist_train = mnist_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
mnist_train = mnist_train.cache()
mnist_train = mnist_train.shuffle(info.splits['train'].num_examples)
mnist_train = mnist_train.prefetch(tf.data.experimental.AUTOTUNE)

mnist_test = mnist_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
mnist_test = mnist_test.cache()
mnist_test = mnist_test.prefetch(tf.data.experimental.AUTOTUNE)

5. We can now apply a mirrored strategy. This strategies goal is to replicate the model across all GPUs on the same machine. Each model is trained on different batches of data and a synchronous training strategy is applied:

In [7]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


This error is weird, but the below code makes it look like we still are on our two virtual GPUs.

6. Check that we have two devices corresponding to the two virtual GPUs created at the start of this recipe:

In [9]:
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

Number of devices: 2


7. We now define the batch size. The batch size given to the dataset is the global batch size. The global batch size is the sum of all batch sizes of every replica. So, we had to compute the global batch size using the number of replicas:

In [10]:
BATCH_SIZE_PER_REPLICA = 128
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

mnist_train = mnist_train.batch(BATCH_SIZE)
mnist_test = mnist_test.batch(BATCH_SIZE)

8. Next we define and compile our model using the mirrored strategy scope. Note that all variables created inside the scope are mirrored across all replicas:

In [11]:
with mirrored_strategy.scope():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(name='FLATTEN'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu', name='D1'))
    model.add(tf.keras.layers.Dense(units=64, activation='relu', name='D2'))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax', name='OUTPUT'))
    
    model.compile(
        optimizer='sgd',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


9. Once the compilation is over, we can fit the previous model as we normally would:

In [12]:
model.fit(mnist_train,
          epochs=10,
          validation_data = mnist_test)

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


235/235 [==============================] - 6s 9ms/step - loss: 1.7546 - accuracy: 0.5502 - val_loss: 1.1243 - val_accuracy: 0.7872
Epoch 2/10
235/235 [==============================] - 1s 3ms/step - loss: 0.8244 - accuracy: 0.8220 - val_loss: 0.6016 - val_accuracy: 0.8572
Epoch 3/10
235/235 [==============================] - 1s 3ms/step - loss: 0.5417 - accuracy: 0.8637 - val_loss: 0.4615 - val_accuracy: 0.8792
Epoch 4/10
235/235 [==============================] - 1s 3ms/step - loss: 0.4472 - accuracy: 0.8798 - val_loss: 0.4020 - val_accuracy: 0.8918
Epoch 5/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3999 - accuracy: 0.8895 - val_loss: 0.3682 - val_accuracy: 0.8980
Epoch 6/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3707 - accuracy: 0.8962 - val_loss: 0.3460 - val_accuracy: 0.9036
Epoch 7/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3503 - accuracy: 0.9008 - val_loss: 0.3286 - val_accuracy: 0.9059
Epoch 8/10
235/23

Using the strategy scope is all that is required to distribute training.

For more information, refer to https://www.tensorflow.org/guide/distributed_training